Copyright (C) rflylab from School of Automation Science and Electrical Engineering, Beihang University.

All Rights Reserved.

This file is an esay example showing how to process data which is originally from ULog.

Details about ULog file is available on [this page](https://docs.px4.io/main/en/dev_log/ulog_file_format.html).

In [1]:
# necessary packages
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R

# the OpenHA
from OpenHA.processing.preprocess import interpolation

The throttle $\sigma\in\left[0,1\right]$ is calculated by the following equation
$$
\sigma=\frac{c-1000}{1000}
$$
where $c$ is the command from ratio which satisfy $c\in\left[1000,2000\right]$.

In [2]:
def pwm2sigma(pwm):
    # throttle
    t = (pwm[:, 1] - 1000) / 1000

    t[t < 0] = 0
    t[t > 1] = 1
    # return with timestamp
    return np.array([pwm[:, 0], t]).T

The relationship between euler angular acceleration and body rotate angular accelaration is

$$\dot{\mathbf{\Theta}}=\mathbf{W}\boldsymbol{\omega}$$

where

$$
\mathbf{W}=\left[\begin{matrix}
            1 & \tan\theta\sin\phi  & \tan\theta\cos\phi  \\
            0 & \cos\phi            & -\sin\phi           \\
            0 & \sin\phi/\cos\theta & \cos\phi/\cos\theta
        \end{matrix}\right].
$$


In [3]:
def mat_pqr2euler(phi, theta):
    '''
    Return the matrix which represents the relationship between angular acceleration and eulur angular acceleration.
    '''
    W = np.array(
        [
            [1, np.tan(theta) * np.sin(phi), np.tan(theta) * np.cos(phi)],
            [0, np.cos(phi), -np.sin(phi)],
            [0, np.sin(phi) / np.cos(theta), np.cos(phi) / np.cos(theta)],
        ]
    )
    return W

In [4]:
# the path of the files
# they are available on https://github.com/rfly-openha/documents/tree/main/2_tutorial/examples
name = 'log_7_2022-5-12-16-22-12'
file_store_location = './' + name
# the list of time
target_timevec = range(26778352, 227309598, 50000)

In [5]:
# position
file_local_posi_name = file_store_location + '_vehicle_local_position_0.csv'
vehicle_local_position = pd.read_csv(file_local_posi_name, header=0).values

# get the data of specific section and then interpolate
z = vehicle_local_position[168 - 1 : 2212, [0, 7]]
vz = vehicle_local_position[168 - 1 : 2212, [0, 13]]
z = interpolation(z[:, 0], z[:, 1], target_timevec)
vz = interpolation(vz[:, 0], vz[:, 1], target_timevec)

In [ ]:
# eulur angles
file_attitude_name = file_store_location + '_vehicle_attitude_0.csv'
vehicle_attitude = pd.read_csv(file_attitude_name, header=0).values

# in SciPy, the real part of the quaternion is in that last
quat = vehicle_attitude[211 - 1 : 4300, [3, 4, 5, 2]]
# convert quaternion to euler angles
euler = R.from_quat(quat).as_euler('zyx', False)
yaw, pitch, roll = euler.T
yaw = interpolation(vehicle_attitude[211 - 1 : 4300, 0], yaw, target_timevec)
pitch = interpolation(vehicle_attitude[211 - 1 : 4300, 0], pitch, target_timevec)
roll = interpolation(vehicle_attitude[211 - 1 : 4300, 0], roll, target_timevec)

In [ ]:
# actuator output
file_actuator_name = file_store_location + '_actuator_outputs_0.csv'
actuator_output = pd.read_csv(file_actuator_name, header=0).values
opt1 = actuator_output[174 - 1 : 2219, [0, 2]]
opt2 = actuator_output[174 - 1 : 2219, [0, 3]]
opt3 = actuator_output[174 - 1 : 2219, [0, 4]]
opt4 = actuator_output[174 - 1 : 2219, [0, 5]]
# convert command to sigma
sigma1 = pwm2sigma(opt1)
sigma1 = interpolation(sigma1[:, 0], sigma1[:, 1], target_timevec)
sigma2 = pwm2sigma(opt2)
sigma2 = interpolation(sigma2[:, 0], sigma2[:, 1], target_timevec)
sigma3 = pwm2sigma(opt3)
sigma3 = interpolation(sigma3[:, 0], sigma3[:, 1], target_timevec)
sigma4 = pwm2sigma(opt4)
sigma4 = interpolation(sigma4[:, 0], sigma4[:, 1], target_timevec)

In [ ]:
# angular acceleration of the aircraft in the body frame
file_sensor_combined_name = file_store_location + '_sensor_combined_0.csv'
sensor_combined = pd.read_csv(file_sensor_combined_name, header=0).values
x_rate = sensor_combined[4186 - 1 : 54607, [0, 1]]
x_rate = interpolation(x_rate[:, 0], x_rate[:, 1], target_timevec)
y_rate = sensor_combined[4186 - 1 : 54607, [0, 2]]
y_rate = interpolation(y_rate[:, 0], y_rate[:, 1], target_timevec)
z_rate = sensor_combined[4186 - 1 : 54607, [0, 3]]
z_rate = interpolation(z_rate[:, 0], z_rate[:, 1], target_timevec)

In [ ]:
# catch as a large matrix
euler_re = np.array([roll, pitch, yaw])
z_re = z
vz_re = vz
omega_re = np.array([x_rate, y_rate, z_rate])
sigma_re = np.array([sigma1, sigma2, sigma3, sigma4])

# compute the euler angular acceleration
length = len(target_timevec)
dot_euler_re = np.zeros((3, length))

for j in range(length):
    W = mat_pqr2euler(euler_re[0, j], euler_re[1, j])
    dot_euler_re[:, (j,)] = np.matmul(W, omega_re[:, (j,)])

Finally you need to store all the needed variables as a `CSV` file.
Just like the `log_7_2022-5-12-16-22-12_lossfactor.csv`.